In [1]:
import torch
import torch.multiprocessing as mp

In [2]:
from rsnn.memorization.posterior import update_posterior

In [3]:
torch.get_num_threads()

8

In [21]:
L = 16

Vw = torch.rand(16, 500, 500)
mw = torch.rand(16, 500, 1)
Vy = torch.rand(16, 50, 500, 500)
my = torch.rand(16, 50, 500, 1)

Vw1 = Vw.clone()
mw1 = mw.clone()

In [20]:
processes = []
for l in range(L):
    p = mp.Process(target=update, args=(Vw1[l], mw1[l], Vy[l], my[l]))
    p.start()
    processes.append(p)
    
for p in processes:
    p.join()

Vw1, mw1

(tensor([[[0.0093, 0.0856, 0.8959,  ..., 0.3229, 0.0050, 0.2991],
          [0.3757, 0.2950, 0.2390,  ..., 0.7166, 0.9087, 0.2110],
          [0.4173, 0.7587, 0.9778,  ..., 0.3732, 0.4460, 0.2493],
          ...,
          [0.7837, 0.3873, 0.8348,  ..., 0.0890, 0.7380, 0.4008],
          [0.1422, 0.1747, 0.8332,  ..., 0.7591, 0.0860, 0.3551],
          [0.3824, 0.6763, 0.9244,  ..., 0.4979, 0.2161, 0.5785]],
 
         [[0.5194, 0.2057, 0.8510,  ..., 0.7689, 0.9467, 0.1805],
          [0.1786, 0.3745, 0.3979,  ..., 0.7989, 0.9967, 0.1905],
          [0.2600, 0.7713, 0.4797,  ..., 0.7473, 0.4611, 0.5668],
          ...,
          [0.2963, 0.6086, 0.0309,  ..., 0.3520, 0.4640, 0.6138],
          [0.2502, 0.2286, 0.8082,  ..., 0.7695, 0.1375, 0.1478],
          [0.5059, 0.2397, 0.2012,  ..., 0.9128, 0.6786, 0.8245]],
 
         [[0.8229, 0.5161, 0.1907,  ..., 0.6970, 0.5639, 0.7346],
          [0.2317, 0.2327, 0.8215,  ..., 0.0810, 0.7781, 0.2180],
          [0.3732, 0.4080, 0.3191,  ...,

In [ ]:
Vw

In [11]:
N = Vy.size(1)
for n in range(N):
    Vw = Vw + Vy[:, n] @ Vw @ Vy[:, n].mT
    mw = mw + Vy[:, n] @ Vw @ my[:, n]
    
Vw

tensor([[[inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         ...,
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        ...,

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]

In [22]:
%%timeit 
torch.set_num_threads(2)
a = torch.rand(8, 2500, 5000)
b = torch.rand(8, 5000, 7500)
c = torch.bmm(a, b)

20.7 s ± 275 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit 
torch.set_num_threads(4)
a = torch.rand(8, 2500, 5000)
b = torch.rand(8, 5000, 7500)
c = torch.bmm(a, b)

12.5 s ± 233 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
torch.set_num_threads(8)
a = torch.rand(8, 2500, 5000)
b = torch.rand(8, 5000, 7500)
c = torch.bmm(a, b)

10.1 s ± 41.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
